In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df=pd.read_csv("../ingest_from_ITS/data/m952_login_2025-05-15_00-00-00_to_2025-06-16_00-00-00.csv",usecols=[
    "@timestamp", "user.id", "event.action","event.its.properties.gold",
    "event.its.properties.diamond","event.its.properties.power_point","event.its.properties.level",
    "event.its.properties.vip_level","event.its.properties.dragon_gold"
    ]) 
df.shape

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_20488\429649107.py:1: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../ingest_from_ITS/data/m952_login_2025-05-15_00-00-00_to_2025-06-16_00-00-00.csv",usecols=[


(1796953, 9)

In [3]:
df=df[~df['user.id'].isnull()]
df.reset_index(drop=True,inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796953 entries, 0 to 1796952
Data columns (total 9 columns):
 #   Column                            Dtype 
---  ------                            ----- 
 0   @timestamp                        object
 1   event.action                      object
 2   event.its.properties.dragon_gold  int64 
 3   event.its.properties.gold         int64 
 4   event.its.properties.power_point  int64 
 5   event.its.properties.level        int64 
 6   event.its.properties.diamond      int64 
 7   event.its.properties.vip_level    int64 
 8   user.id                           object
dtypes: int64(6), object(3)
memory usage: 123.4+ MB


In [ ]:
df.head(3)

,@timestamp,event.action,event.its.properties.dragon_gold,event.its.properties.gold,event.its.properties.power_point,event.its.properties.level,event.its.properties.diamond,event.its.properties.vip_level,user.id
0,2025-05-14T17:00:00.000Z,its_login,171.00,3752910,1425822,47,390.00,3,2504191703569743
1,2025-05-14T17:00:00.000Z,its_login,0.00,287350,36020,20,0.00,0,2505062131237087
2,2025-05-14T17:00:00.000Z,its_login,592.00,7324129,1489372,51,323.00,3,2504102040141995


In [6]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'], errors='coerce', utc=True)

In [7]:
len(df['user.id'].unique())

103302

In [ ]:
# numeric_fields = [
#     "event.its.properties.vip_level",
#     "event.its.properties.dragon_gold",
#     "event.its.properties.diamond",
#     "event.its.properties.gold",
#     "event.its.properties.power_point",
#     "event.its.properties.level"
# ]

# for col in numeric_fields:
#     if col in df.columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce').fillna(-1.0)

In [ ]:
df=df.rename(columns={'user.id':'vopenid'})
df['vopenid']=df['vopenid'].astype(str)

In [ ]:
now = pd.Timestamp("2025-06-16 12:00:00", tz="UTC")

In [ ]:
df=df.sort_values(by=['@timestamp'])

In [ ]:
df['@timestamp'].min(),df['@timestamp'].max()

(Timestamp('2025-05-14 17:00:00+0000', tz='UTC'),
 Timestamp('2025-05-15 13:36:07+0000', tz='UTC'))

In [ ]:
data=df

In [ ]:
# time update
def last_update_time(group):
    last_time = group['@timestamp'].max()
    return {
        'last_update_time': last_time
    }

In [ ]:
# count how many days that user login
def num_day_login(group):
    num_day = group['@timestamp'].dt.date.nunique()
    return {
        "num_day_login":num_day,
    }

In [ ]:
# count how many sessions that user login per day
def num_session_on_day_login(group):
    if group.empty:
        return {
            'avg_login_per_day': 0
        }
    group['date'] = group['@timestamp'].dt.date
    daily_counts = group.groupby('date').size()
    avg = daily_counts.mean()
    return {
        'avg_login_per_day': avg
    }

In [ ]:
# calculate average time amongs logins of user
def avg_between_login(group):
    times = group['@timestamp'].sort_values()
    if len(times) < 2:
        return {
            'avg_between_login': 0
        }
    deltas = times.diff().dropna()
    res=deltas.mean().total_seconds() / 86400
    return {
        'avg_between_login': res
    }

In [ ]:
# the longest streak day that user login
def streak_login(group):
    days = pd.to_datetime(group['@timestamp'].dt.date).sort_values().drop_duplicates()
    streak = 1
    max_streak=1
    for i in range(1, len(days)):
        if (days.iloc[i] - days.iloc[i-1]).days == 1:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 1
    return {
        'streak_login': max_streak
    }

In [ ]:
# how many days that user not login again
def last_since_from_login(group):
    last_time = group['@timestamp'].max()
    last_time = last_time.tz_localize('UTC') if last_time.tzinfo is None else last_time
    res=(now-last_time).days
    return {
        'last_since_from_login': res
    }

In [ ]:
# the popular time that user login
def popular_time_login(group):
    hours = group['@timestamp'].dt.hour
    mode = hours.mode()
    res=mode.iloc[0] if not mode.empty else None
    return {
        'popular_time_login': res
    }

In [ ]:
# std gold
def std_gold_login(group):
    gold_series = group.sort_values('@timestamp')['event.its.properties.gold']
    if len(gold_series) < 2:
        std_gold = 0.0  
    else:
        std_gold = gold_series.std()
    
    return {
        'gold_std_login': std_gold
    }

In [ ]:
# std diamond
def std_diamond_login(group):
    diamond_series = group.sort_values('@timestamp')['event.its.properties.diamond']
    if len(diamond_series) < 2:
        std_diamond = 0.0  
    else:
        std_diamond = diamond_series.std()
    
    return {
        'diamond_std_login': std_diamond
    }

In [ ]:
# std silkcoin
def std_silkcoin_login(group):
    silkcoin_series = group.sort_values('@timestamp')['event.its.properties.dragon_gold']
    if len(silkcoin_series) < 2:
        std_silkcoin = 0.0  
    else:
        std_silkcoin = silkcoin_series.std()
    
    return {
        'silkcoin_std_login': std_silkcoin
    }

In [ ]:
# the last powerpoint of user
def last_login_powerpoint(group):
    latest = group.sort_values('@timestamp', ascending=False).iloc[0]
    return {
        'powerpoint_last_login': latest['event.its.properties.power_point']
    }

In [ ]:
# slope gold
def slope_gold_login(group):
    group_sorted = group.sort_values('@timestamp')
    gold_values = group_sorted['event.its.properties.gold'].values

    n = len(gold_values)
    if n < 2:
        return { 'gold_slope_login': 0.0 }

    x = np.arange(n)
    y = gold_values

    mean_x = x.mean()
    mean_y = y.mean()

    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sum((x - mean_x) ** 2)

    slope = numerator / denominator if denominator != 0 else 0.0

    return {
        'gold_slope_login': slope
    }

In [ ]:
# slope diamond
def slope_diamond_login(group):
    group_sorted = group.sort_values('@timestamp')
    diamond_values = group_sorted['event.its.properties.diamond'].values

    n = len(diamond_values)
    if n < 2:
        return { 'diamond_slope_login': 0.0 }

    x = np.arange(n)
    y = diamond_values

    mean_x = x.mean()
    mean_y = y.mean()

    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sum((x - mean_x) ** 2)

    slope = numerator / denominator if denominator != 0 else 0.0

    return {
        'diamond_slope_login': slope
    }

In [ ]:
# slope silkcoin
def slope_silkcoin_login(group):
    group_sorted = group.sort_values('@timestamp')
    silkcoin_values = group_sorted['event.its.properties.dragon_gold'].values

    n = len(silkcoin_values)
    if n < 2:
        return { 'silkcoin_slope_login': 0.0 }

    x = np.arange(n)
    y = silkcoin_values

    mean_x = x.mean()
    mean_y = y.mean()

    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sum((x - mean_x) ** 2)

    slope = numerator / denominator if denominator != 0 else 0.0

    return {
        'silkcoin_slope_login': slope
    }

In [ ]:
# level of user
def last_login_level(group):
    latest = group.sort_values('@timestamp', ascending=False).iloc[0]
    return {
        'level_last_login': latest['event.its.properties.level']
    }

In [ ]:
# vip of user
def last_login_vip(group):
    latest = group.sort_values('@timestamp', ascending=False).iloc[0]
    return {
        'vip_last_login': latest['event.its.properties.vip_level']
    }

In [ ]:
features_function=[
   last_update_time,
   num_day_login,
   num_session_on_day_login,
   avg_between_login,
   streak_login,
   last_since_from_login,
   popular_time_login,
   std_gold_login,
   std_diamond_login,
   std_silkcoin_login,
   slope_gold_login,
   slope_diamond_login,
   slope_silkcoin_login,
   last_login_powerpoint,
   last_login_level,
   last_login_vip,
]

In [ ]:
def extract_group_features(group):
    result={}
    for func in features_function:
        try:
            result.update(func(group))
        except Exception as e:
            print(f"Error at feature: {func}. Error: {e}")
    return pd.Series(result)

df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()
df_res

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_20620\3747869921.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()


,vopenid,last_update_time,num_day_login,avg_login_per_day,avg_between_login,streak_login,last_since_from_login,popular_time_login,gold_std_login,diamond_std_login,silkcoin_std_login,gold_slope_login,diamond_slope_login,silkcoin_slope_login,powerpoint_last_login,level_last_login,vip_last_login
0,1000,2025-05-15 02:32:35+00:00,1,3.00,0.01,1,32,2,10439797.10,0.00,0.00,0.00,0.00,0.00,2,0,-1.00
1,1024,2025-05-15 04:48:25+00:00,2,32.00,0.01,2,32,4,11783130.48,0.00,0.00,51604.98,0.00,0.00,1,2,-1.00
2,1025,2025-05-15 02:11:19+00:00,1,1.00,0.00,1,32,2,0.00,0.00,0.00,0.00,0.00,0.00,2,0,-1.00
3,1030,2025-05-15 04:49:49+00:00,2,4.00,0.07,2,32,17,8631798.50,0.00,0.00,792708.85,0.00,0.00,1,0,-1.00
4,1036,2025-05-15 04:15:19+00:00,2,3.50,0.07,2,32,2,4410333.92,0.00,0.00,-156223.32,0.00,0.00,1,0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59564,900,2025-05-15 13:35:59+00:00,2,152.50,0.00,2,31,17,830014.79,1.46,0.50,-7574.55,0.01,0.00,2,1105309683,-1.00
59565,904,2025-05-15 09:04:59+00:00,1,2.00,0.02,1,32,8,0.00,0.00,0.00,0.00,0.00,0.00,1,1105309683,-1.00
59566,922,2025-05-14 17:41:01+00:00,1,1.00,0.00,1,32,17,0.00,0.00,0.00,0.00,0.00,0.00,2,1,-1.00
59567,959,2025-05-15 04:20:22+00:00,2,1.00,0.44,2,32,4,0.00,0.00,0.00,0.00,0.00,0.00,1,13,-1.00


In [ ]:
df_res['vopenid'] = df_res['vopenid'].astype(str).str.strip()

In [ ]:
df_res.to_csv("data/login_transform.csv",index=False,header=True)

In [ ]:
df_res['vopenid'].nunique()

59569

In [ ]:
df_res

,vopenid,last_update_time,num_day_login,avg_login_per_day,avg_between_login,streak_login,last_since_from_login,popular_time_login,gold_std_login,diamond_std_login,silkcoin_std_login,gold_slope_login,diamond_slope_login,silkcoin_slope_login,powerpoint_last_login,level_last_login,vip_last_login
0,1000,2025-05-15 02:32:35+00:00,1,3.00,0.01,1,32,2,10439797.10,0.00,0.00,0.00,0.00,0.00,2,0,-1.00
1,1024,2025-05-15 04:48:25+00:00,2,32.00,0.01,2,32,4,11783130.48,0.00,0.00,51604.98,0.00,0.00,1,2,-1.00
2,1025,2025-05-15 02:11:19+00:00,1,1.00,0.00,1,32,2,0.00,0.00,0.00,0.00,0.00,0.00,2,0,-1.00
3,1030,2025-05-15 04:49:49+00:00,2,4.00,0.07,2,32,17,8631798.50,0.00,0.00,792708.85,0.00,0.00,1,0,-1.00
4,1036,2025-05-15 04:15:19+00:00,2,3.50,0.07,2,32,2,4410333.92,0.00,0.00,-156223.32,0.00,0.00,1,0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59564,900,2025-05-15 13:35:59+00:00,2,152.50,0.00,2,31,17,830014.79,1.46,0.50,-7574.55,0.01,0.00,2,1105309683,-1.00
59565,904,2025-05-15 09:04:59+00:00,1,2.00,0.02,1,32,8,0.00,0.00,0.00,0.00,0.00,0.00,1,1105309683,-1.00
59566,922,2025-05-14 17:41:01+00:00,1,1.00,0.00,1,32,17,0.00,0.00,0.00,0.00,0.00,0.00,2,1,-1.00
59567,959,2025-05-15 04:20:22+00:00,2,1.00,0.44,2,32,4,0.00,0.00,0.00,0.00,0.00,0.00,1,13,-1.00
